In [1]:
import modulos as md
import sys
import os
import constantes as cons
import numpy as np 
import pandas as pd
symbol='STXUSDT'

lista=md.lista_de_monedas()
for symbol in lista:
    try:
        data = md.obtiene_historial(symbol)
        data = md.estrategia_bb(data)
        resultado = md.backtesting(data)
        if resultado[6]>1.5:
            print(f"{symbol} - retorno: {md.truncate(resultado[6],2)}% - trades: {resultado[17]}")
    except Exception as falla:
        _, _, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print(f"Symbol: {symbol} - Error: {str(falla)} - line: {str(exc_tb.tb_lineno)} - file: {str(fname)}\n")
        pass  


C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


EOSUSDT - retorno: 6.43% - trades: 6
XLMUSDT - retorno: 1.76% - trades: 5
XTZUSDT - retorno: 2.82% - trades: 7
IOTAUSDT - retorno: 1.54% - trades: 9
BATUSDT - retorno: 1.89% - trades: 6
THETAUSDT - retorno: 7.57% - trades: 7
ZRXUSDT - retorno: 2.83% - trades: 5
Falla leyendo...
Symbol: DEFIUSDT - Error: 'NoneType' object has no attribute 'ema20' - line: 13 - file: 2244024983.py



c:\LUCHO\personal\repopersonal\luchocrypto\FENIX\modulos.py:302: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, Fenix, cash=1000)


RUNEUSDT - retorno: 1.51% - trades: 9
SUSHIUSDT - retorno: 6.73% - trades: 9
SRMUSDT - retorno: 2.25% - trades: 5
STORJUSDT - retorno: 3.86% - trades: 6
FTMUSDT - retorno: 1.82% - trades: 7
ENJUSDT - retorno: 3.87% - trades: 7
CTKUSDT - retorno: 1.92% - trades: 5
1INCHUSDT - retorno: 2.46% - trades: 10
MANAUSDT - retorno: 2.52% - trades: 15
ALICEUSDT - retorno: 2.43% - trades: 12
Falla leyendo...
Symbol: 1000SHIBUSDT - Error: 'NoneType' object has no attribute 'ema20' - line: 13 - file: 2244024983.py

Falla leyendo...
Symbol: BTCDOMUSDT - Error: 'NoneType' object has no attribute 'ema20' - line: 13 - file: 2244024983.py

Falla leyendo...
Symbol: 1000XECUSDT - Error: 'NoneType' object has no attribute 'ema20' - line: 13 - file: 2244024983.py

CELOUSDT - retorno: 3.73% - trades: 7
Falla leyendo...
Symbol: FOOTBALLUSDT - Error: 'NoneType' object has no attribute 'ema20' - line: 13 - file: 2244024983.py

Falla leyendo...
Symbol: 1000LUNCUSDT - Error: 'NoneType' object has no attribute 'ema